## basic test

In [68]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [69]:
#-- REAL WORLD configuration, which is attached to the physics engine
# accessing this directly would be cheating!
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

In [70]:
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [71]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/pandasTable.g")

In [72]:
#-- using the viewer, you can view configurations or paths
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [73]:
#-- the following is the simulation loop
tau = .01

for t in range(5):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    [y,J] = C.evalFeature(ry.FS.position, ["R_gripper"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ [0.,0.,-1e-1];

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

## doing things relative to an object

In [74]:
# add a new frame to the MODEL configuration
# (Perception will later have to do exactly this: add perceived objects to the model)
capsule = C.addFrame("capsule")
box     = C.addFrame("box")

In [75]:
# set frame parameters, associate a shape to the frame, 
capsule.setPosition([.8,0,1.5])
capsule.setQuaternion([1,0,.5,0])
capsule.setShape(ry.ST.capsule, [.2,.02])
capsule.setColor([1,0,1])


box.setPosition([-.8,0,1.5])
box.setQuaternion([1,0,.5,0])
box.setShape(ry.ST.box, [0.1,0.1,0.1])
box.setColor([1,1,0])


V.setConfiguration(C)
#V.recopyMeshes(C) #this is rarely necessary, only when you change meshes within C

In [76]:
#-- the following is the simulation loop
tau = .01

#[y_itself ,J_itself]  = C.evalFeature(ry.FS.position, ["L_gripperCenter"])

for t in range(500):
    y = []
    J = []
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    
    [y_capsule,J_capsule] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "capsule"])
    [y_box    ,J_box]     = C.evalFeature(ry.FS.positionDiff, ["L_gripperCenter", "box"])
    [y_LGrip  ,J_LGrip]   = C.evalFeature(ry.FS.position, ["L_gripperCenter"])
        
    # merge all tasks
    
    # reach capsule
    y = y_capsule
    J = J_capsule
    
    # stay close to initial position
    #y  = np.concatenate((y ,  (y_LGrip - y_itself)))
    #J = np.vstack((J,  J_LGrip))
    
    # reach box
    y  = np.concatenate((y ,  y_box/tau))
    J = np.vstack((J,  J_box/tau))
    
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);
    
    # stop Left gripper lifting
    vel[0] = 0
    
    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

How could you align the gripper for a proper grasp?

In [67]:
S=0
V=0
C=0
RealWorld=0

In [56]:
vel

array([ 1.20530195e-03,  8.32129104e-04,  7.37533044e-03,  2.65195821e-03,
        2.91730930e-03,  5.43299331e-04,  4.19913932e-03, -5.33468827e-09,
        0.00000000e+00, -9.93030449e-04,  1.08702300e-02, -3.96479269e-03,
        7.57881804e-03, -5.53363185e-04,  7.43794935e-03, -6.42306208e-09,
        0.00000000e+00])